In [ ]:
import pickle
import boto3
import pandas as pd
from io import StringIO, BytesIO
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy.stats import normaltest, ttest_ind

from helper_scripts.utility_functions import *

## Set AWS Credentials

Do not put quotation marks around the value.

In [ ]:
%env BUCKET_NAME=aws_bucket_name

## Connect to AWS

In [ ]:
# Create a session using the default credentials (IAM role attached to the instance)
session = boto3.Session()

# Create an S3 client
s3 = session.client('s3')

# Specify your bucket name
bucket_name = os.getenv('BUCKET_NAME')


## Read Pickle File Containing Results from Human Labels (Worst Overall Classification Model)

In [ ]:
link_to_pickle_file = '/data/outputs_50/model_outputs/clsf-v3/human_labels_clsf_all_scores.pickle'
clsf_human = read_pickle_from_s3(s3, bucket_name, link_to_pickle_file)
clsf_human

## Read Pickle File Containing Results from ML Captions (Best Overall Classification Model)

In [ ]:
link_to_pickle_file = '/data/outputs_50/model_outputs/clsf-v3/ml_capt_clsf_all_scores.pickle'
clsf_ml_capt = read_pickle_from_s3(s3, bucket_name, link_to_pickle_file)
clsf_ml_capt

In [ ]:
sample1 = clsf_human['overall_weighted']
np.mean(sample1)

In [ ]:
sample2 = clsf_ml_capt['overall_weighted']
np.mean(sample2)

# Summary Statistics


In [ ]:
results = pd.DataFrame()
results['A'] = sample1
results['B'] = sample2
results

Descriptive Stats

In [ ]:
print(results.describe())

Box and Whisker Plot

In [ ]:
results.boxplot()
pyplot.show()


Histogram

In [ ]:
# histogram
results.hist()
pyplot.show()

# Normality Test

In [ ]:
value, p = normaltest(sample1)
print(value, p)
if p >= 0.05:
 print('It is likely that result1 is normal')
else:
 print('It is unlikely that result1 is normal')

In [ ]:
value, p = normaltest(sample2)
print(value, p)
if p >= 0.05:
 print('It is likely that result2 is normal')
else:
 print('It is unlikely that result2 is normal')

# Compare Means for Gaussian Results

In [ ]:
# load results1
values1 = sample1
# load results2
values2 = sample2
# calculate the significance
value, pvalue = ttest_ind(values1, values2, equal_var=False)
print(value, pvalue)
if pvalue > 0.05:
	print('Samples are likely drawn from the same distributions (fail to reject H0)')
else:
	print('Samples are likely drawn from different distributions (reject H0)')

# END